In [ ]:
#Import generic datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
#Import all required Keras modeling datasets
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, BatchNormalization
from keras.models import Model
from keras.layers import ReLU 

In [ ]:
dir = os.getcwd()
print("Current Dir: ", dir)
train_dir = os.path.join(dir, "train")
print("Train Dir: ", train_dir)
train_files = os.listdir(train_dir)
train_labels = []
for file in train_files:
    identifier = file.split('.')[0]
    if identifier ==  'dog':
        train_labels.append(1)
    elif identifier == 'cat':
        train_labels.append(0)
    else:
        train_labels.append('Error')
train_df_orig = pd.DataFrame({'img': train_files, 'label': train_labels})

In [ ]:
print(train_df_orig)

                img  label
0         cat.0.jpg      0
1         cat.1.jpg      0
2        cat.10.jpg      0
3       cat.100.jpg      0
4      cat.1000.jpg      0
...             ...    ...
24995  dog.9995.jpg      1
24996  dog.9996.jpg      1
24997  dog.9997.jpg      1
24998  dog.9998.jpg      1
24999  dog.9999.jpg      1

[25000 rows x 2 columns]


In [ ]:
#All images need to be the same size
import cv2
def resize_images(input_dir, output_dir, target_size=(500, 500)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for filename in os.listdir(input_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(input_dir, filename))
            if img.shape[:2] != target_size:
                resized_img = cv2.resize(img, target_size)
                cv2.imwrite(os.path.join(output_dir, filename), resized_img)

In [ ]:
train_dir_resize = os.path.join(dir, "train_dir_resize")

In [ ]:
#Only needs to be run on our first take with the dataset
resize_images(train_dir, train_dir_resize)  

In [ ]:
#Only needs to be run on our first take with the dataset
test_dir = os.path.join(dir, "test1")
print("Test Dir: ", test_dir)
test_dir_resize = os.path.join(dir, "test_dir_resize")

Test Dir:  c:\Users\Connor\Documents\AAI628\Final\test1


In [ ]:
#Only needs to be run on our first take with the dataset
resize_images(test_dir, test_dir_resize)  

In [ ]:
train_files_resized = os.listdir(train_dir_resize)
train_labels_resized = []
for file in train_files_resized:
    identifier = file.split('.')[0]
    if identifier ==  'dog':
        train_labels_resized.append(1)
    elif identifier == 'cat':
        train_labels_resized.append(0)
    else:
        train_labels_resized.append('Error')
train_df_resized = pd.DataFrame({'img': train_files_resized, 'label': train_labels_resized})

In [ ]:
print(train_df_resized)

                img  label
0         cat.0.jpg      0
1         cat.1.jpg      0
2        cat.10.jpg      0
3       cat.100.jpg      0
4      cat.1000.jpg      0
...             ...    ...
24995  dog.9995.jpg      1
24996  dog.9996.jpg      1
24997  dog.9997.jpg      1
24998  dog.9998.jpg      1
24999  dog.9999.jpg      1

[25000 rows x 2 columns]


In [ ]:
#Model parameters
img_height = 500
img_width = 500
img_size = (500, 500)
channels = 3
number_of_classes = 2

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(500, 500, channels)))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.2))
model.add(Dense(1))

model.add(Activation('sigmoid'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 498, 498, 32)      896       
                                                                 
 activation_6 (Activation)   (None, 498, 498, 32)      0         
                                                                 
 batch_normalization_5 (Batc  (None, 498, 498, 32)     128       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 496, 496, 32)      9248      
                                                                 
 activation_7 (Activation)   (None, 496, 496, 32)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 248, 248, 32)     0         
 2D)                                                  

In [ ]:
from keras.optimizers import Adam
#We need to use binary cross entropy, as output will be either a 1 or 0
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
image_fit_dir = train_dir_resize
batch_size = 32
image_generator = ImageDataGenerator(rescale=1.0/255.0)  # Rescale pixel values between 0 and 1
target_size = (img_width, img_height)
fit_df = train_df_resized
fit_df['label'] = fit_df['label'].astype('string')

In [ ]:
train_data = image_generator.flow_from_dataframe(
    fit_df,
    directory=image_fit_dir,
    x_col='img',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

Found 25000 validated image filenames belonging to 2 classes.


In [ ]:
#Fit the model to my training data
epochs = 10
model.fit(train_data, epochs = epochs)